In [1]:
import grib2io

## **Opening GRIB2 files**

Note that nothing has changed between v1 and v2 wrt to opening GRIB2 files and the same attributes exist.

In [2]:
g = grib2io.open("gfs.t00z.pgrb2.0p25.f006")
print(g)

mode = rb
name = gfs.t00z.pgrb2.0p25.f006
messages = 743
current_message = 0
size = 544359352
closed = False
variables = ('4LFTX', 'ABSV', 'ACPCP', 'ALBDO', 'APCP', 'APTMP', 'CAPE', 'CFRZR', 'CICEP', 'CIN', 'CLMR', 'CNWAT', 'CPOFP', 'CPRAT', 'CRAIN', 'CSNOW', 'CWAT', 'CWORK', 'DLWRF', 'DPT', 'DSWRF', 'DZDT', 'FLDCP', 'FRICV', 'GFLUX', 'GRLE', 'GUST', 'HCDC', 'HGT', 'HINDEX', 'HLCY', 'HPBL', 'ICAHT', 'ICEC', 'ICEG', 'ICETK', 'ICETMP', 'ICMR', 'LAND', 'LCDC', 'LFT X', 'LHTFL', 'MCDC', 'MSLET', 'O3MR', 'PEVPR', 'PLPL', 'POT', 'PRATE', 'PRES', 'PRMSL', 'PWAT', 'REFC', 'REFD', 'RH', 'RWMR', 'SFCR', 'SHTFL', 'SNMR', 'SNOD', 'SOILL', 'SOILW', 'SOTYP', 'SPFH', 'SUNSD', 'TCDC', 'TMAX', 'TMIN', 'TMP', 'TOZNE', 'TSOIL', 'U-GWD', 'UFLX', 'UGRD', 'ULWRF', 'USTM', 'USWRF', 'V-GWD', 'VEG', 'VFLX', 'VGRD', 'VIS', 'VRATE', 'VSTM', 'VVEL', 'VWSH', 'WATR', 'WEASD', 'WILT')
levels = ('0-0.1 m underground', '0.01 mb', '0.02 mb', '0.04 mb', '0.07 mb', '0.1 mb', '0.1-0.4 m underground', '0.2 mb', '0.33-1 sig

## **Investigation of the file object index**

Underneath the hood, the indexing has changed significantly. There no longer is GRIB2-specific metadata from messages, but just information about location
and size of messages.  We are now storing the GRIB2 message object in the index in key 'msg'.

One might think that storing all of these objects in the index would negatively impact performance, but the new GRIB2 message object is extremely lightweight (more on that later) and acts like a templated container of metadata.  The data section is not a part of the GRIB2 message object.

In [3]:
print(g._index.keys())

dict_keys(['offset', 'bitmap_offset', 'data_offset', 'size', 'data_size', 'submessageOffset', 'submessageBeginSection', 'isSubmessage', 'messageNumber', 'msg'])


## **Reading GRIB2 Messages from file**

### Use read(), seek(), tell()

Can use custom implementations of read(), seek(), and tell().  These operate on units of GRIB2 messages, but bytes.

Put the file object pointer to the 100th message and read the next 15 messages (from message 100).

In [52]:
g.seek(100)
print('GRIB2 file poistion: ',g.tell())
msgs = g.read(15)
print('Number of messages read: ',len(msgs))
for msg in msgs:
    print(msg)

GRIB2 file poistion:  100
Number of messages read:  15
100:d=2022-10-28 00:00:00:UGRD:U-Component of Wind (m s-1):1 mb:6:00:00
101:d=2022-10-28 00:00:00:VGRD:V-Component of Wind (m s-1):1 mb:6:00:00
102:d=2022-10-28 00:00:00:ABSV:Absolute Vorticity (s-1):1 mb:6:00:00
103:d=2022-10-28 00:00:00:O3MR:Ozone Mixing Ratio (kg kg-1):1 mb:6:00:00
104:d=2022-10-28 00:00:00:HGT:Geopotential Height (gpm):2 mb:6:00:00
105:d=2022-10-28 00:00:00:TMP:Temperature (K):2 mb:6:00:00
106:d=2022-10-28 00:00:00:RH:Relative Humidity (%):2 mb:6:00:00
107:d=2022-10-28 00:00:00:SPFH:Specific Humidity (kg kg-1):2 mb:6:00:00
108:d=2022-10-28 00:00:00:VVEL:Vertical Velocity (Pressure) (Pa s-1):2 mb:6:00:00
109:d=2022-10-28 00:00:00:DZDT:Vertical Velocity (Geometric) (m s-1):2 mb:6:00:00
110:d=2022-10-28 00:00:00:UGRD:U-Component of Wind (m s-1):2 mb:6:00:00
111:d=2022-10-28 00:00:00:VGRD:V-Component of Wind (m s-1):2 mb:6:00:00
112:d=2022-10-28 00:00:00:ABSV:Absolute Vorticity (s-1):2 mb:6:00:00
113:d=2022-10-28 0

### Use index, slice, or key notation on the file object

**_IMPORTANT: In grib2io v2.0, you no longer have to specify a 2nd index when select 1 message !!_**

In [50]:
# Index
msg = g[50]
print(msg)

50:d=2022-10-28 00:00:00:UGRD:U-Component of Wind (m s-1):0.07 mb:6:00:00


**_IMPORTANT:  In grib2io v2.0, a 0th message now exists !!_**

In [51]:
msg = g[0]
print(msg)

0:d=2022-10-28 00:00:00:PRMSL:Pressure Reduced to MSL (Pa):mean sea level:6:00:00


In [6]:
# Slice
msgs = g[80:90]
for msg in msgs:
    print(msg)

80:d=2022-10-28 00:00:00:UGRD:U-Component of Wind (m s-1):0.4 mb:6:00:00
81:d=2022-10-28 00:00:00:VGRD:V-Component of Wind (m s-1):0.4 mb:6:00:00
82:d=2022-10-28 00:00:00:ABSV:Absolute Vorticity (s-1):0.4 mb:6:00:00
83:d=2022-10-28 00:00:00:O3MR:Ozone Mixing Ratio (kg kg-1):0.4 mb:6:00:00
84:d=2022-10-28 00:00:00:HGT:Geopotential Height (gpm):0.7 mb:6:00:00
85:d=2022-10-28 00:00:00:TMP:Temperature (K):0.7 mb:6:00:00
86:d=2022-10-28 00:00:00:RH:Relative Humidity (%):0.7 mb:6:00:00
87:d=2022-10-28 00:00:00:SPFH:Specific Humidity (kg kg-1):0.7 mb:6:00:00
88:d=2022-10-28 00:00:00:VVEL:Vertical Velocity (Pressure) (Pa s-1):0.7 mb:6:00:00
89:d=2022-10-28 00:00:00:DZDT:Vertical Velocity (Geometric) (m s-1):0.7 mb:6:00:00


In [7]:
# Key - where string is the GRIB2 shortName attribute
msgs = g['CAPE']
for msg in msgs:
    print(msg)

623:d=2022-10-28 00:00:00:CAPE:Convective Available Potential Energy (J kg-1):surface:6:00:00
709:d=2022-10-28 00:00:00:CAPE:Convective Available Potential Energy (J kg-1):180-0 mb above ground:6:00:00
722:d=2022-10-28 00:00:00:CAPE:Convective Available Potential Energy (J kg-1):90-0 mb above ground:6:00:00
724:d=2022-10-28 00:00:00:CAPE:Convective Available Potential Energy (J kg-1):255-0 mb above ground:6:00:00


### Selection of GRIB2 messages based on attributes

Use any Grib2Message object attribute name as a keyword for the file object select() method. **NOTE:** The select() method will always return a list.

#### Example 1: select temperature message (TMP) that use product definition template number of 8.

In [8]:
msgs = g.select(shortName='TMP',productDefinitionTemplateNumber=8)
for msg in msgs:
    print(msg)
    print('\t',msg.duration,msg.statisticalProcess)

646:d=2022-10-28 00:00:00:TMP:Temperature (K):low cloud top level:6:00:00
	 6:00:00 0 - Average
647:d=2022-10-28 00:00:00:TMP:Temperature (K):middle cloud top level:6:00:00
	 6:00:00 0 - Average
648:d=2022-10-28 00:00:00:TMP:Temperature (K):high cloud top level:6:00:00
	 6:00:00 0 - Average


#### Example 2: select U-wind messages (UGRD) at geometric heights above ground.

In [9]:
msgs = g.select(shortName='UGRD',typeOfFirstFixedSurface=103)
for msg in msgs:
    print(msg)
    print('\t',msg.typeOfFirstFixedSurface)

587:d=2022-10-28 00:00:00:UGRD:U-Component of Wind (m s-1):10 m above ground:6:00:00
	 103 - ['Specified Height Level Above Ground', 'm']
674:d=2022-10-28 00:00:00:UGRD:U-Component of Wind (m s-1):20 m above ground:6:00:00
	 103 - ['Specified Height Level Above Ground', 'm']
676:d=2022-10-28 00:00:00:UGRD:U-Component of Wind (m s-1):30 m above ground:6:00:00
	 103 - ['Specified Height Level Above Ground', 'm']
678:d=2022-10-28 00:00:00:UGRD:U-Component of Wind (m s-1):40 m above ground:6:00:00
	 103 - ['Specified Height Level Above Ground', 'm']
680:d=2022-10-28 00:00:00:UGRD:U-Component of Wind (m s-1):50 m above ground:6:00:00
	 103 - ['Specified Height Level Above Ground', 'm']
685:d=2022-10-28 00:00:00:UGRD:U-Component of Wind (m s-1):80 m above ground:6:00:00
	 103 - ['Specified Height Level Above Ground', 'm']
688:d=2022-10-28 00:00:00:UGRD:U-Component of Wind (m s-1):100 m above ground:6:00:00
	 103 - ['Specified Height Level Above Ground', 'm']


## **Handling Grib2Message Object and Attributes**

Lets use 500mb HGT as an example

In [10]:
msg = g.select(shortName='HGT',level='500 mb')[0]
print(msg)

348:d=2022-10-28 00:00:00:HGT:Geopotential Height (gpm):500 mb:6:00:00


The Grib2Message object provides a wgrib2-line 1-liner string for `__str__` (as shown above).  The Grib2Message object `__repr__` will provide all metadata

In [11]:
msg

Section 0: discipline = 0 - Meteorological Products
Section 1: originatingCenter = 7 - US National Weather Service - NCEP (WMC)
Section 1: originatingSubCenter = 0 - None
Section 1: masterTableInfo = 2 - Version Implemented on 4 November 2003
Section 1: localTableInfo = 1 - Number of local table version used.
Section 1: significanceOfReferenceTime = 1 - Start of Forecast
Section 1: year = 2022
Section 1: month = 10
Section 1: day = 28
Section 1: hour = 0
Section 1: minute = 0
Section 1: second = 0
Section 1: refDate = 2022-10-28 00:00:00
Section 1: validDate = 2022-10-28 06:00:00
Section 1: productionStatus = 0 - Operational Products
Section 1: typeOfData = 1 - Forecast Products
Section 3: gridDefinitionTemplateNumber = 0 - Latitude/Longitude
Section 3: shapeOfEarth = 6 - Earth assumed spherical with radius = 6,371,229.0 m
Section 3: earthRadius = 6371229.0
Section 3: earthMajorAxis = None
Section 3: earthMinorAxis = None
Section 3: resolutionAndComponentFlags = [0, 0, 1, 1, 0, 0, 0, 0

**IMPORTANT:** These metadata attributes are no longer set at object instantiation as they were in grib2io v1.x.  In v2.0, these attributes are first class variables of the Grib2Message object or class variables that have been inherited from the template classes. Then each class variable are defined further as descriptor classes with special implementations of the `__get__` and `__set__` protocols that allows these attributes to provide an on-the-fly decoding of the GRIB2 integer, table-driven metadata values.

Technically speaking, the only data stored in the Grib2Message object are the unpacked section arrays containing the GRIB2 integer metadata.

In [12]:
print('Section 0: ',msg.section0)
print('Section 1: ',msg.section1)
print('Section 2: ',msg.section2)
print('Section 3: ',msg.section3)
print('Section 4: ',msg.section4)
print('Section 5: ',msg.section5)

Section 0:  [1196575042          0          0          2     811530]
Section 1:  [   7    0    2    1    1 2022   10   28    0    0    0    0    1]
Section 2:  b''
Section 3:  [        0   1038240         0         0         0         6         0
         0         0         0         0         0      1440       721
         0        -1  90000000         0        48 -90000000 359750000
    250000    250000         0]
Section 4:  [    0     0     3     5     2     0    96     0     0     1     6   100
     0 50000   255     0     0]
Section 5:  [   1038240          3 1223386786          1          2         13
          0          1          0 1649987994 4294967295      28103
          0          4          1          1         41          7
          2          2]


The Grib2Message class is dynamically created and is unique to GRIB2's grid definition, production definition, and data representation template numbers. These templates contain metadata attributes unique to the definition of the given template.

Here we can use the class method resolution order (`__mro__`) to show the various classes that make up the Grib2Message class. 

In [13]:
for c in msg.__class__.__mro__:
    print(c)

<class 'grib2io._grib2io.Grib2Message.__new__.<locals>.Msg'>
<class 'grib2io._grib2io._Grib2Message'>
<class 'grib2io.templates.GridDefinitionTemplate0'>
<class 'grib2io.templates.ProductDefinitionTemplate0'>
<class 'grib2io.templates.DataRepresentationTemplate3'>
<class 'object'>


The template classes listed above contain the attributes as class variables that are unique to its template.

You will notice that the `ProductDefinitionTemplate0` is blank.  This is because it contains the common attributes used by the rest of the PTDNs, so they are defined in class `_Grib2Message`.

In [14]:
print(grib2io.templates.GridDefinitionTemplate0._attrs)
print(grib2io.templates.ProductDefinitionTemplate0._attrs)
print(grib2io.templates.DataRepresentationTemplate3._attrs)

['latitudeFirstGridpoint', 'longitudeFirstGridpoint', 'latitudeLastGridpoint', 'longitudeLastGridpoint', 'gridlengthXDirection', 'gridlengthYDirection']
[]
['refValue', 'binScaleFactor', 'decScaleFactor', 'nBitsPacking', 'typeOfMissingValue', 'priMissingValue', 'secMissingValue', 'nGroups', 'refGroupWidth', 'nBitsGroupWidth', 'refGroupLength', 'groupLengthIncrement', 'lengthOfLastGroup', 'nBitsScaledGroupLength', 'spatialDifferenceOrder', 'nBytesSpatialDifference']


Now lets take a look at how the metadata attributes are linked to their origin locations in a given section.  Here we will look at the Grib2Message `refDate` attribute, whose components are stored in Section 1.

To reiterate, `msg.refDate` does not hold a value that would be a `datetime.datetime` object.  The `refDate` attribute is defined as a descriptor class, specifically `grib2io.templates.RefDate()`, whose `__get__` protocol returns a `datetime.datetime` object generated from the date components in section 1.

In [15]:
print(msg.section1)
print(msg.refDate)
print(msg.year,msg.month,msg.day,msg.hour,msg.minute,msg.second)

[   7    0    2    1    1 2022   10   28    0    0    0    0    1]
2022-10-28 00:00:00
2022 10 28 0 0 0


To demonstrate that these attributes are dynamically created, lets change the year and minute. We do this by setting the attributes to new values.  You will see that values have changed in section1 and other attributes.

In [16]:
msg.year = 2023
msg.minute = 55
print(msg.section1)
print(msg.refDate)

[   7    0    2    1    1 2023   10   28    0   55    0    0    1]
2023-10-28 00:55:00


**IMPORTANT:** In grib2io v2.0, all date and time attributes are datetime or timedelta objects.  Let use precipitation as an example.

In [17]:
msg = g['APCP'][0]
print(msg)
print(msg.refDate,type(msg.refDate))
print(msg.validDate,type(msg.validDate))
print(msg.leadTime,type(msg.leadTime))
print(msg.duration,type(msg.duration))

595:d=2022-10-28 00:00:00:APCP:Total Precipitation (kg m-2):surface:6:00:00
2022-10-28 00:00:00 <class 'datetime.datetime'>
2022-10-28 06:00:00 <class 'datetime.datetime'>
6:00:00 <class 'datetime.timedelta'>
6:00:00 <class 'datetime.timedelta'>


This allows for datetime arithmetic

A `pdy` attribute is provided

In [18]:
print(msg.refDate + msg.leadTime)
print(msg.validDate)
print(msg.refDate+msg.leadTime==msg.validDate)
print(msg.pdy)

2022-10-28 06:00:00
2022-10-28 06:00:00
True
20221028


The Grib2Message object provides a method, `attrs_by_section()` to get attribute names and values (if desired) for a given GRIB2 section.

In [24]:
for k,v in msg.attrs_by_section(3,values=True).items():
    print(k,v)

gridDefinitionTemplateNumber 0 - Latitude/Longitude
shapeOfEarth 6 - Earth assumed spherical with radius = 6,371,229.0 m
earthRadius 6371229.0
earthMajorAxis None
earthMinorAxis None
resolutionAndComponentFlags [0, 0, 1, 1, 0, 0, 0, 0]
ny 721
nx 1440
scanModeFlags [0, 0, 0, 0]
latitudeFirstGridpoint 90.0
longitudeFirstGridpoint 0.0
latitudeLastGridpoint -90.0
longitudeLastGridpoint 359.75
gridlengthXDirection 0.25
gridlengthYDirection -0.25


Any GRIB2 metadata attribute that is a code value means that it is associated with a code table. That attribute is likely to be of type `grib2io.templates.Grib2Metadata` which is a class to hold the code value, the definition given its lookup table.

Let use the GRIB2 metadata attribute `discipline` as an example.

In [28]:
print(type(msg.discipline))
print(msg.discipline)
print(msg.discipline.value)
print(msg.discipline.definition)
print(msg.discipline.table)

<class 'grib2io.templates.Grib2Metadata'>
0 - Meteorological Products
0
Meteorological Products
0.0


### A Quick Detour to NCEP GRIB2 Tables

grib2io contains the [NCEP GRIB2 Tables](https://www.nco.ncep.noaa.gov/pmb/docs/grib2/grib2_doc/) stored as python dictionaries in `grib2io.tables`.  You are able to access them directly,

In [81]:
print(grib2io.tables.section0.table_0_0)
print(grib2io.tables.section4.table_4_6)

{'0': 'Meteorological Products', '1': 'Hydrological Products', '2': 'Land Surface Products', '3': 'Satellite Remote Sensing Products', '4': 'Space Weather Products', '5-9': 'Reserved', '10': 'Oceanographic Products', '11-19': 'Reserved', '20': 'Health and Socioeconomic Impacts', '21-191': 'Reserved', '192-254': 'Reserved for Local Use', '255': 'Missing'}
{'0': 'Unperturbed High-Resolution Control Forecast', '1': 'Unperturbed Low-Resolution Control Forecast', '2': 'Negatively Perturbed Forecast', '3': 'Positively Perturbed Forecast', '4': 'Multi-Model Forecast', '5-191': 'Reserved', '192-254': 'Reserved for Local Use', '192': 'Perturbed Ensemble Member', '255': 'Missing'}


However, it is more useful to use the various helper functions in `grib2io.tables` to view the NCEP GRIB2 code tables and lookup values.

In [87]:
for k,v in grib2io.tables.get_table('0.0').items():
    print(f'{k}: {v}')

0: Meteorological Products
1: Hydrological Products
2: Land Surface Products
3: Satellite Remote Sensing Products
4: Space Weather Products
5-9: Reserved
10: Oceanographic Products
11-19: Reserved
20: Health and Socioeconomic Impacts
21-191: Reserved
192-254: Reserved for Local Use
255: Missing


In [88]:
print('Value 2 from table 4.6 means',grib2io.tables.get_value_from_table(2,'4.6'))

Value 2 from table 4.6 means Negatively Perturbed Forecast


In [91]:
discipline, parmcat, parmnum = 0, 1, 10
varinfo = grib2io.tables.get_varinfo_from_table(discipline,parmcat,parmnum)
print(f'Discipline value of {discipline} with paramater category and number of {parmcat} and {parmnum} defines variable {varinfo}')

Discipline value of 0 with paramater category and number of 1 and 10 defines variable ['Convective Precipitation', 'kg m-2', 'ACPCP']


Some metadata attributes exist for more easier understanding of information from the Grib2Message.

The `level` attribute will provide a layer/level from that is **_exactly the same as wgrib2_** and is generated by `grib2io.tables.get_wgrib2_level_string()`.

In [38]:
print(msg.level)
print(msg.units) # Units of the variable
print(msg.unitOfFirstFixedSurface) # Units of the level (or layer)

500 mb
gpm
Pa


## **Unpacking Data from Grib2Message Objects**

Lets go back to the 500 mb HGT example.

Data are accessible via the `data` attribute.

**IMPORTANT:** If you unfamiliar with Python/NumPy, by default, multi-dimensional array data are ordered in row-major order.  This is how C/C++ order array data in memory.  Fortran orders array data in column-major order. So 2D array will be ordered (ny, nx).

In [60]:
msg = g.select(shortName='HGT',level='500 mb')[0]
print(msg)
hgtdata = msg.data
print(hgtdata)
print(hgtdata.shape)

348:d=2023-10-28 00:55:00:HGT:Geopotential Height (gpm):500 mb:6:00:00
[[5097.0503 5097.0503 5097.0503 ... 5097.0503 5097.0503 5097.0503]
 [5095.2505 5095.2505 5095.2505 ... 5095.2505 5095.2505 5095.2505]
 [5092.9307 5092.9307 5092.9307 ... 5092.9106 5092.9106 5092.9106]
 ...
 [5086.5303 5086.5107 5086.5107 ... 5086.5703 5086.5503 5086.5303]
 [5086.4106 5086.4106 5086.3906 ... 5086.4307 5086.4307 5086.4106]
 [5085.7905 5085.7905 5085.7905 ... 5085.7905 5085.7905 5085.7905]]
(721, 1440)


## **Geospatial Information from Grib2Message Objects**

GRIB2 Section 3 contains all of the geospatial information about the message.

In [61]:
print(msg.section3)
for k,v in msg.attrs_by_section(3,values=True).items():
    print(f'{k}: {v}')

[        0   1038240         0         0         0         6         0
         0         0         0         0         0      1440       721
         0        -1  90000000         0        48 -90000000 359750000
    250000    250000         0]
gridDefinitionTemplateNumber: 0 - Latitude/Longitude
shapeOfEarth: 6 - Earth assumed spherical with radius = 6,371,229.0 m
earthRadius: 6371229.0
earthMajorAxis: None
earthMinorAxis: None
resolutionAndComponentFlags: [0, 0, 1, 1, 0, 0, 0, 0]
ny: 721
nx: 1440
scanModeFlags: [0, 0, 0, 0]
latitudeFirstGridpoint: 90.0
longitudeFirstGridpoint: 0.0
latitudeLastGridpoint: -90.0
longitudeLastGridpoint: 359.75
gridlengthXDirection: 0.25
gridlengthYDirection: -0.25


grib2io also provides the coorindate system in terms of PROJ parameters.

In [62]:
print(msg.projParameters)

{'a': 6371229.0, 'b': 6371229.0, 'proj': 'longlat'}


### Generating Latitude and Longitude Value

Latitude and Longitude values can be generated via `grid()` or `latlons()` methods.

In [63]:
lats, lons = msg.grid()
print(lats)
print(lons)
msg.grid() == msg.latlons() # latlons() is an alias of grid()

[[ 90.    90.    90.   ...  90.    90.    90.  ]
 [ 89.75  89.75  89.75 ...  89.75  89.75  89.75]
 [ 89.5   89.5   89.5  ...  89.5   89.5   89.5 ]
 ...
 [-89.5  -89.5  -89.5  ... -89.5  -89.5  -89.5 ]
 [-89.75 -89.75 -89.75 ... -89.75 -89.75 -89.75]
 [-90.   -90.   -90.   ... -90.   -90.   -90.  ]]
[[0.0000e+00 2.5000e-01 5.0000e-01 ... 3.5925e+02 3.5950e+02 3.5975e+02]
 [0.0000e+00 2.5000e-01 5.0000e-01 ... 3.5925e+02 3.5950e+02 3.5975e+02]
 [0.0000e+00 2.5000e-01 5.0000e-01 ... 3.5925e+02 3.5950e+02 3.5975e+02]
 ...
 [0.0000e+00 2.5000e-01 5.0000e-01 ... 3.5925e+02 3.5950e+02 3.5975e+02]
 [0.0000e+00 2.5000e-01 5.0000e-01 ... 3.5925e+02 3.5950e+02 3.5975e+02]
 [0.0000e+00 2.5000e-01 5.0000e-01 ... 3.5925e+02 3.5950e+02 3.5975e+02]]


True

**NEW** In grib2io v2.0, computed lats and lons are stored so they do not have to regenerated if you call the methods from another message with the same coordinate system.  This is determined by taking the SHA-1 of section 3 and storing that as a key in a dictionary where lats and lons are the values.

In [64]:
print(msg._sha1_section3)
print(grib2io._grib2io._latlon_datastore)

e8e4ad7e93f0f476804d00043204634232ebe9aa
{'e8e4ad7e93f0f476804d00043204634232ebe9aa': (array([[ 90.  ,  90.  ,  90.  , ...,  90.  ,  90.  ,  90.  ],
       [ 89.75,  89.75,  89.75, ...,  89.75,  89.75,  89.75],
       [ 89.5 ,  89.5 ,  89.5 , ...,  89.5 ,  89.5 ,  89.5 ],
       ...,
       [-89.5 , -89.5 , -89.5 , ..., -89.5 , -89.5 , -89.5 ],
       [-89.75, -89.75, -89.75, ..., -89.75, -89.75, -89.75],
       [-90.  , -90.  , -90.  , ..., -90.  , -90.  , -90.  ]]), array([[0.0000e+00, 2.5000e-01, 5.0000e-01, ..., 3.5925e+02, 3.5950e+02,
        3.5975e+02],
       [0.0000e+00, 2.5000e-01, 5.0000e-01, ..., 3.5925e+02, 3.5950e+02,
        3.5975e+02],
       [0.0000e+00, 2.5000e-01, 5.0000e-01, ..., 3.5925e+02, 3.5950e+02,
        3.5975e+02],
       ...,
       [0.0000e+00, 2.5000e-01, 5.0000e-01, ..., 3.5925e+02, 3.5950e+02,
        3.5975e+02],
       [0.0000e+00, 2.5000e-01, 5.0000e-01, ..., 3.5925e+02, 3.5950e+02,
        3.5975e+02],
       [0.0000e+00, 2.5000e-01, 5.0000e-01, .

## **Interpolation**

**NEW** In grib2io v2.0 is the ability to perform spatial interpolation from one grid to another by interfacing to the [NCEPLIBS-ip](https://github.com/NOAA-EMC/NCEPLIBS-ip) library.

The Grib2Message object already contains the necessary information about the input grid. So all we need to go here is to define the output grid (i.e. the grid that we want to interpolate to).  We do that by specifcy the GRIB2 Grid Definition Template Number and Template values.  Here we will interpolate to the NBM CONUS 2.5km Lambert Conformal grid.

Here we place the template number and template into a `Grib2GridDef` object.

In [65]:
nbm_co_gdtn = 30
nbm_co_gdt = [        1,         0,   6371200,       255,       255,       255,
                    255,      2345,      1597,  19229000, 233723400,        48,
               25000000, 265000000,   2539703,   2539703,         0,        80,
               25000000,  25000000, -90000000,         0]
grid_def_out = grib2io.Grib2GridDef(nbm_co_gdtn,nbm_co_gdt)
print(grid_def_out.gdtn)
print(grid_def_out.gdt)

30
[1, 0, 6371200, 255, 255, 255, 255, 2345, 1597, 19229000, 233723400, 48, 25000000, 265000000, 2539703, 2539703, 0, 80, 25000000, 25000000, -90000000, 0]


To interpolate a Grib2Message object, call the `interpolate()` method.  The return from the method is a new Grib2Message object.  All metadata remain the same except for section 3 metadata.

**NOTE 1: The interpolation option can be a string or numeric value determined by the NCEPLIBS-ip library.**

**NOTE 2: The message number of interpolated messages is -1. This is becasue this message does not originate from a physical file.  Any Grib2Message object created from scatch will have message number of -1 until it is associated with a file object.**

In [76]:
new = msg.interpolate('bilinear',grid_def_out)
print(new)
print(new.section3)
for k,v in msg.attrs_by_section(3,values=True).items():
    print(f'{k}: {v}')
print(new.data)

-1:d=2023-10-28 00:55:00:HGT:Geopotential Height (gpm):500 mb:6:00:00
[        0   3744965         0       255        30         1         0
   6371200       255       255       255       255      2345      1597
  19229000 233723400        48  25000000 265000000   2539703   2539703
         0        80  25000000  25000000 -90000000         0]
gridDefinitionTemplateNumber: 0 - Latitude/Longitude
shapeOfEarth: 6 - Earth assumed spherical with radius = 6,371,229.0 m
earthRadius: 6371229.0
earthMajorAxis: None
earthMinorAxis: None
resolutionAndComponentFlags: [0, 0, 1, 1, 0, 0, 0, 0]
ny: 721
nx: 1440
scanModeFlags: [0, 0, 0, 0]
latitudeFirstGridpoint: 90.0
longitudeFirstGridpoint: 0.0
latitudeLastGridpoint: -90.0
longitudeLastGridpoint: 359.75
gridlengthXDirection: 0.25
gridlengthYDirection: -0.25
[[5885.6235 5885.614  5885.574  ... 5849.2563 5849.1816 5849.145 ]
 [5885.62   5885.6123 5885.5825 ... 5849.261  5849.1973 5849.1465]
 [5885.6167 5885.611  5885.592  ... 5849.278  5849.207  5849.

## **Creating Grib2Message Object from Scratch**

**NOTE:** As of 20230113, We are still working a few bugs related to this.

## **Writing Grib2Message Objects to a New File**

**NOTE:** As of 20230113, We are still working a few bugs related to this.